## Customer Churn Prediction_ Model Training & Comparison

In [1]:
# STEP 10 — Model Training & Comparison (Churn-Focused)
# 10.1 Load Prepared Data (From Step 9)
import pandas as pd

X_train_scaled = pd.read_csv("../data/processed/X_train_scaled.csv")
X_test_scaled  = pd.read_csv("../data/processed/X_test_scaled.csv")

y_train = pd.read_csv("../data/processed/y_train.csv").squeeze()
y_test  = pd.read_csv("../data/processed/y_test.csv").squeeze()

X_train_smote = pd.read_csv("../data/processed/X_train_smote.csv")
y_train_smote = pd.read_csv("../data/processed/y_train_smote.csv").squeeze()

In [2]:
# 10.2 Define Evaluation Metrics (Churn-Relevant)
from sklearn.metrics import (
    roc_auc_score,
    recall_score,
    precision_score,
    f1_score
)

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    return {
        "ROC_AUC": roc_auc_score(y_test, y_prob),
        "Recall_Churn": recall_score(y_test, y_pred),
        "Precision_Churn": precision_score(y_test, y_pred),
        "F1": f1_score(y_test, y_pred)
    }

In [3]:
#10.3 Model 1 — Logistic Regression (Baseline, Explainable)
# 10.3.1 With Class Weights
from sklearn.linear_model import LogisticRegression

log_reg_w = LogisticRegression(
    class_weight="balanced",
    max_iter=1000,
    random_state=42
)

log_reg_w.fit(X_train_scaled, y_train)

lr_weighted_results = evaluate_model(
    log_reg_w, X_test_scaled, y_test
)

lr_weighted_results


{'ROC_AUC': 0.8440505220166237,
 'Recall_Churn': 0.7371007371007371,
 'Precision_Churn': 0.4665629860031104,
 'F1': 0.5714285714285714}

In [4]:
# 10.3.2 With SMOTE
log_reg_smote = LogisticRegression(
    max_iter=1000,
    random_state=42
)

log_reg_smote.fit(X_train_smote, y_train_smote)

lr_smote_results = evaluate_model(
    log_reg_smote, X_test_scaled, y_test
)

lr_smote_results

{'ROC_AUC': 0.8324811714642224,
 'Recall_Churn': 0.5528255528255528,
 'Precision_Churn': 0.6267409470752089,
 'F1': 0.587467362924282}

In [5]:
# 10.4 Model 2 — Random Forest (Non-Linear)
# 10.4.1 With Class Weights
from sklearn.ensemble import RandomForestClassifier

rf_w = RandomForestClassifier(
    n_estimators=300,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)

rf_w.fit(X_train_scaled, y_train)

rf_weighted_results = evaluate_model(
    rf_w, X_test_scaled, y_test
)

rf_weighted_results

{'ROC_AUC': 0.8520222842256739,
 'Recall_Churn': 0.4275184275184275,
 'Precision_Churn': 0.7699115044247787,
 'F1': 0.5497630331753555}

In [6]:
# 10.4.2 With SMOTE
rf_smote = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    n_jobs=-1
)

rf_smote.fit(X_train_smote, y_train_smote)

rf_smote_results = evaluate_model(
    rf_smote, X_test_scaled, y_test
)

rf_smote_results

{'ROC_AUC': 0.8494326375682308,
 'Recall_Churn': 0.515970515970516,
 'Precision_Churn': 0.6687898089171974,
 'F1': 0.5825242718446602}

In [7]:
# 10.5 Consolidated Model Comparison Table
results_df = pd.DataFrame.from_dict(
    {
        "LogReg_ClassWeight": lr_weighted_results,
        "LogReg_SMOTE": lr_smote_results,
        "RF_ClassWeight": rf_weighted_results,
        "RF_SMOTE": rf_smote_results
    },
    orient="index"
)

results_df.sort_values("Recall_Churn", ascending=False)


,ROC_AUC,Recall_Churn,Precision_Churn,F1
LogReg_ClassWeight,0.844051,0.737101,0.466563,0.571429
LogReg_SMOTE,0.832481,0.552826,0.626741,0.587467
RF_SMOTE,0.849433,0.515971,0.668790,0.582524
RF_ClassWeight,0.852022,0.427518,0.769912,0.549763


In [8]:
# Best model is LogReg_ClassWeight because it has the highest Recall_Churn score of 0.70, indicating it is most effective at identifying customers who will churn.
!pip install joblib


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
# Save the best model
import joblib
import os

os.makedirs("../models", exist_ok=True)

joblib.dump(
    log_reg_w,
    "../models/best_churn_model.pkl"
)

['../models/best_churn_model.pkl']

In [10]:
# verification
loaded_model = joblib.load("../models/best_churn_model.pkl")
loaded_model.predict(X_test_scaled[:5])


array([0, 0, 0, 0, 0], dtype=int64)

In [12]:
# 10.6 Final Evaluation on Test Set 
from sklearn.metrics import classification_report, roc_auc_score

y_pred = loaded_model.predict(X_test_scaled)
y_prob = loaded_model.predict_proba(X_test_scaled)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))


              precision    recall  f1-score   support

           0       0.92      0.78      0.85      1593
           1       0.47      0.74      0.57       407

    accuracy                           0.78      2000
   macro avg       0.69      0.76      0.71      2000
weighted avg       0.83      0.78      0.79      2000

ROC-AUC: 0.8440505220166237
